# Long COVID coding in primary care.

This OpenSAFELY report is a routine update of our peer-review paper published in the _British Journal of General Practice_ on the [Clinical coding of long COVID in English primary care: a federated analysis of 58 million patient records in situ using OpenSAFELY](https://doi.org/10.3399/BJGP.2021.0301). 

It is a routine update of the analysis described in the paper. The data requires careful interpretation and there are a number of caveats. Please read the full detail about our methods and discussionis and the full analytical methods on this routine report are [available on GitHub](https://github.com/opensafely/long-covid).

OpenSAFELY is a new secure analytics platform for electronic patient records built on behalf of NHS England to deliver urgent academic and operational research during the pandemic. You can read more about [OpenSAFELY on our website](https://github.com/opensafely/long-covid).

In [ ]:
from datetime import datetime
from IPython.display import display, Markdown
import pandas as pd
pd.set_option("display.max_rows", 200)
pd.options.display.float_format = "{:,.1f}".format
float_formatter = "{:,.0f}".format
import lib

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
tpp = lib.tpp_emis_table_format("output", {"0": "Missing"})
total_tpp = tpp.loc["sex",["No long COVID", "Long COVID"]].sum()
all_total_tpp = total_tpp.sum()
emis = lib.tpp_emis_table_format("emis", {"RGN11NM": "region","0": "Missing","6": "Missing"})
total_emis = emis.loc["sex",["No long COVID", "Long COVID"]].sum()
all_total_emis = total_emis.sum()
totals = tpp[["No long COVID", "Long COVID"]] + emis[["No long COVID", "Long COVID"]]

#display(Markdown(f"### Report last updated **{datetime.today().strftime('%d %b %Y')}**"))

## Results

### Counts and rates of long COVID coding stratified by demographic variable
This is equivalent to Table 2 from [the paper](https://doi.org/10.3399/BJGP.2021.0301)

In [ ]:
total, table_2 = lib.get_table_2([tpp, emis, totals])
display(Markdown(f"There were {float_formatter(total[('Totals','Long COVID')])} people who have been given a diagnostic code for long COVID to date. Counts for each software system are:"))
print(total.loc[(slice(None), "Long COVID")])
display(Markdown(f"The overall rate of long COVID coding in the population was {round(total[('Totals','Rate per 100,000')]/2,1)} per 100,000 people. Rates for each software system are:"))
print(total.loc[(slice(None), "Rate per 100,000")]/2)
table_2

### Volume of code use in individual practices
Stratified by the electronic health record provider of the practice (TPP/SystmOne or EMIS).

In [ ]:
lib.code_use_per_week_graph()

### Use of long COVID codes over time
Stratified by the electronic health record provider of the practice (TPP/SystmOne or EMIS). Reporting lag may affect recent dates.

In [ ]:
lib.practice_distribution_graph()

### Total use of each individual long COVID related code
This is distinct from the above table in that it counts all coded events, including where patients have been coded more than once.

In [ ]:
total, codes_tpp = lib.smoosh_codes_tables()
total = total[(slice(None), "Total records")]
pd.set_option('display.max_colwidth', 200)
display(Markdown(f"There were {float_formatter(total['Total'])} long COVID codes used in total. Counts for each software system were:"))
print(total)
codes_tpp

### Characteristics of the cohort
This is equivalent to Table 1 in [the paper](https://doi.org/10.3399/BJGP.2021.0301)

In [ ]:
total, table_1 = lib.get_table_1([tpp, emis, totals])
display(Markdown(f"There were {float_formatter(total[('Totals','Patient count')])} people in the cohort in total. In practices that use TPP software, there were {float_formatter(total[('TPP','Patient count')])}, while in practices that use EMIS software, there were {float_formatter(total[('EMIS','Patient count')])} people."))
display(table_1)